In [1]:
import pandas as pd
from sqlalchemy import create_engine, URL
from tqdm import tqdm

# Initiate Database Connection

In [ ]:
url_object = URL.create(
    drivername="",
    username="",
    password="",
    host="",
    port="",
    database=""
)
engine = create_engine(url_object)

# Extract random Patents from database for Neural Network training (see 03b_Filter_Random_Patents_Classifier.py)

In [4]:
df_non_cleantech_patents_metadata = pd.read_csv('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_non_cleantech_all_title_abstract.csv')

In [5]:
len(df_non_cleantech_patents_metadata)

2577214

# Extract Patent Metadata (Authors, Citations)

In [6]:
# Write df_non_cleantech_patents_metadata to database
df_non_cleantech_patents_metadata.to_sql('temp_non_cleantech_metadata', engine, if_exists='replace', index=False)

214

In [7]:
df_patstat_person_id = pd.read_sql_query("""
    SELECT tp.appln_id::TEXT, pa.person_id
    FROM temp_non_cleantech_metadata AS tp
    JOIN tls207_pers_appln AS pa ON tp.appln_id::TEXT = pa.appln_id
""", con=engine)
df_patstat_person_id = df_patstat_person_id.drop_duplicates(subset=['appln_id', 'person_id'])

In [8]:
df_patstat_citations = pd.read_sql_query("""
    SELECT c.pat_publn_id::text, c.cited_pat_publn_id::text, p.appln_id::text
    FROM tls212_citation AS c
    JOIN tls211_pat_publn AS p ON c.pat_publn_id = p.pat_publn_id
    WHERE p.appln_id IN (SELECT appln_id::text FROM temp_non_cleantech_metadata)
""", con=engine)
df_patstat_citations = df_patstat_citations.drop_duplicates(subset=['pat_publn_id', 'cited_pat_publn_id', 'appln_id'])
df_patstat_citations = df_patstat_citations[df_patstat_citations['cited_pat_publn_id'].isin(df_patstat_citations['pat_publn_id'])]

In [9]:
df_patstat_citations = df_patstat_citations.rename(columns={"appln_id": "pat_appln_id"})
df_patstat_citations = pd.merge(df_patstat_citations, df_patstat_citations[['pat_publn_id', 'pat_appln_id']].rename(columns={'pat_appln_id': 'cited_pat_appln_id'}), left_on='cited_pat_publn_id', right_on='pat_publn_id', how='inner')
df_patstat_citations = df_patstat_citations[['pat_publn_id_x', 'cited_pat_publn_id', 'pat_appln_id', 'cited_pat_appln_id']]
df_patstat_citations = df_patstat_citations.rename(columns={'pat_publn_id_x': 'pat_publn_id'})

In [10]:
df_patstat_person_id.to_csv('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_non_cleantech_all_patstat_person_id.csv', index=False)
df_patstat_citations.to_csv('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_non_cleantech_all_patstat_citations.csv', index=False)